<a href="https://colab.research.google.com/github/Michael-Jimenez-C/CieCom2/blob/main/MapaCII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import folium
import numpy as np

In [65]:
fc=lambda a: 2*np.arctan2(pow(a,1/2),pow(1-a,1/2))
fa=lambda lat1,long1,lat2,long2: pow(np.sin((lat2-lat1)/2),2)+np.cos(lat1)*np.cos(lat2)*pow(np.sin((long2-long1)/2),2)
fd=lambda A,B: 6378E1*fc(fa(A[0],A[1],B[0],B[1]))

In [77]:
class Nodo:
  def __init__(self,nombre,coordenadas):
    self.nombre=nombre
    self.coordenadas=coordenadas
    self.conexiones=[]
    self.status=0#0 no visitado, 1 visitado
    self.auxdist=np.infty
  
  def conectar(self,other):
    #d=lambda A,B: pow(pow(A[0]-B[0],2)+pow(A[1]-B[1],2),1/2)
    if other not in self.conexiones:
      self.conexiones.append((other,fd(self.coordenadas,other.coordenadas)))
    if self not in other.conexiones:
      other.conexiones.append((self,fd(self.coordenadas,other.coordenadas)))
  

  def distancia(self,nombreNodo):
    for i in self.conexiones:
      if i[0].nombre==nombreNodo:
        return  [i[0],i[1]]
    distancia=np.infty
    d=[]
    for i in self.conexiones:
      if i[0].status==1:
        continue
      self.status=1
      k=i[0].distancia(nombreNodo)
      if k!=None:
        if i[1]+k[1]<distancia:
          distancia=i[1]+k[1]
          d=[i[0]]
          d.extend(([k[0]] if type(k[0])!=list else k[0]))
    self.status=0
    return [d,distancia]

  def dijkstra(self):
    pass


  def __repr__(self):
    return self.nombre

In [67]:
A=Nodo("A",(0,1))
B=Nodo("B",(1,2))
C=Nodo("C",(2,1))
D=Nodo("D",(3,3))
E=Nodo("E",(3,2))
F=Nodo("F",(4,0))
G=Nodo("G",(4,2))
H=Nodo("H",(4,3))

In [68]:
A.conectar(B)
A.conectar(C)
B.conectar(D)
B.conectar(E)
B.conectar(C)
C.conectar(F)
C.conectar(G)
D.conectar(H)
E.conectar(G)
G.conectar(H)

In [69]:
F.distancia("E")

127560.0
208851.16779549548
127560.0
63780.0
183145.62435985586


[[C, B, E], 319803.7100929145]

#Mapa

In [70]:
puntos={"Bogota":([4.61,-74.082], "Soacha,La calera,Mosquera".split(",")),
        "Soacha":([4.583,-74.217], "Chipaque,Fusagasuga,La Mesa,Bogota".split(",")),
        "Madrid":([4.733,-74.267], "Facatativa,La Mesa,Mosquera,Funza".split(",")),
        "Facatativa":([4.817,-74.367], "Madrid,La Mesa,La Vega,Villeta".split(",")),
        "Fusagasuga":([4.333,-74.35], "Soacha".split(",")),
        "Funza":([4.717,-74.217], "Mosquera,Madrid,Cota".split(",")),
        "Chia":([4.85,-74.05], "Bogota,Cota,Gachancipa,Zipaquira".split(",")),
        "Zipaquira":([5.02836,-73.9992], "Gachancipa,Chia".split(",")),
        "Girardot":([4.3,-74.8], "Tocaima,Fusagasuga".split(",")),
        "Villeta":([5.017,-74.467], "La Vega,Facatativa".split(",")),
        "Macheta":([5.07983,-73.6072], "Gachancipa".split(",")),
        "La Vega":([4.99946,-74.3399], "Villeta,Bogota".split(",")),
        "Tocaima":([4.45903,-74.634], "Girardot,La Mesa".split(",")),
        "Mosquera":([4.70612,-74.2307], "Madrid,Funza,Bogota".split(",")),
        "La Mesa":([4.633,-74.467], "Soacha,Tocaima".split(",")),
        "Cota":([4.817,-74.1], "Chia,Funza".split(",")),
        "Gachancipa":([4.99228,-73.8712], "Macheta,Chia".split(",")),
        "La calera":([4.717,-73.967], "Bogota,Chipaque".split(",")),
        "Choachi":([4.52748,-73.9257], "Chipaque,La calera".split(",")),
        "Chipaque":([4.44294,-74.0445], "Choachi,Bogota".split(","))}

In [108]:
m=folium.Map(location=[4.61,-74.082], zoom_start=10)
for i in puntos:
  folium.Marker(
      puntos[i][0], popup=f"<i>{i}</i>"
  ).add_to(m)

for i in puntos:
  c=puntos[i]
  for j in c[1]:
    folium.PolyLine([c[0],puntos[j][0]], tooltip="").add_to(m)

m

In [78]:
nodos={i:Nodo(i,puntos[i][0]) for i in puntos}

In [79]:
for i in puntos:
  c=puntos[i]
  for j in c[1]:
    nodos[i].conectar(nodos[j])

In [111]:
origen="Fusagasuga"
destino="Macheta"
ruta=nodos[origen].distancia(destino)
print(f"ruta{ruta[0]} Distancia{ruta[1]}")


ruta[0].insert(0,nodos[origen])

for j in range(len(ruta[0])-1):
  folium.PolyLine([ruta[0][j].coordenadas,ruta[0][j+1].coordenadas], color="red").add_to(m)
m

ruta[Soacha, Bogota, Chia, Gachancipa, Macheta] Distancia50380.99315697961
